In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import numpy as np
import pandas as pd
from IPython.display import display

import sys
sys.path.insert(0, '../src')
from WMSDTransformer import WMSDTransformer

In [3]:
dataset = pd.read_csv("../data/sez.csv", index_col=0)
preference_directions = ["cost", "cost", "gain", "gain", "gain"]
alternative_names = dataset.index.tolist()
criteria_names = dataset.columns.tolist()
alternative_count = len(alternative_names)
weights = np.array([6.06, 26.95, 20.02, 16.53, 30.44])

display(dataset)

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
SEZ Code,,,,,
KAM,540.8285,2.5573,7530,60,0.5551
KOS,2201.2549,7.1334,32400,180,22.9849
KRA,949.6604,4.2404,29580,189,1.3730
LEG,1341.1473,5.1318,15294,86,7.6145
LOD,1754.6376,13.3187,33401,209,7.4028
MIE,1723.9743,7.8381,34992,268,4.9560
POM,2246.2929,10.4816,24893,173,1.4791
SLU,910.1585,1.5923,3478,79,0.7615
STA,707.9814,1.7909,6829,56,0.7010


In [4]:
wmsd_transformer = WMSDTransformer("R")
result_df = wmsd_transformer.fit_transform(
    X=dataset,
    weights=weights.tolist(),
    objectives=preference_directions,
    expert_range=None,
)
result_df = result_df.copy()
result_df["Rank"] = result_df["R"].to_numpy().argsort()[::-1].argsort() + 1
display(result_df.sort_values(by="Rank"))

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result,Mean,Std,R,Rank
SEZ Code,,,,,,,,,
KOS,0.026408,0.527468,0.917751,0.584906,1.000000,0.511000,0.154095,0.715430,1
TAR,0.221691,0.498704,0.547757,0.655660,0.787582,0.416627,0.090155,0.624094,2
MIE,0.306262,0.467373,1.000000,1.000000,0.196208,0.335386,0.216194,0.507296,3
KRA,0.760281,0.774176,0.828267,0.627358,0.036465,0.313381,0.235065,0.484773,4
LEG,0.530733,0.698160,0.374944,0.141509,0.314733,0.279956,0.127373,0.435913,5
SLU,0.783443,1.000000,0.000000,0.108491,0.009202,0.220499,0.297905,0.412212,6
LOD,0.288282,0.000000,0.949515,0.721698,0.305295,0.242178,0.222863,0.411375,7
STA,0.901990,0.983064,0.106334,0.000000,0.006505,0.221202,0.292498,0.411306,8
KAM,1.000000,0.917707,0.128578,0.018868,0.000000,0.211243,0.273909,0.397995,9


# Direct method

In [5]:
alternative = "LEG"
current_rank = result_df.loc[alternative, "Rank"]
print(f"\nProcessing {alternative}, current position in ranking: {current_rank}")
for criterion in criteria_names:
    print()
    for target_rank in range(current_rank - 1, 0, -1):
        solution = wmsd_transformer.improvement("improvement_single_feature", alternative, target_rank, feature_to_change=criterion)
        if solution is not None:
            print(f"{alternative} {current_rank} --> {target_rank} changing {criterion: <21}:", end=" ")
            print(np.round(solution[criterion].item(), 2))
        else:
            print(f"Not possible to achieve rank {target_rank} by modifying {criterion} only")


Processing LEG, current position in ranking: 5

Not possible to achieve rank 4 by modifying Total Area only
Not possible to achieve rank 3 by modifying Total Area only
Not possible to achieve rank 2 by modifying Total Area only
Not possible to achieve rank 1 by modifying Total Area only

LEG 5 --> 4 changing Capital Expenditures : -2.07
LEG 5 --> 3 changing Capital Expenditures : -3.21
Not possible to achieve rank 2 by modifying Capital Expenditures only
Not possible to achieve rank 1 by modifying Capital Expenditures only

LEG 5 --> 4 changing Number of Jobs       : 10371.62
LEG 5 --> 3 changing Number of Jobs       : 15432.68
Not possible to achieve rank 2 by modifying Number of Jobs only
Not possible to achieve rank 1 by modifying Number of Jobs only

LEG 5 --> 4 changing Business Permits     : 104.26
LEG 5 --> 3 changing Business Permits     : 151.2
Not possible to achieve rank 2 by modifying Business Permits only
Not possible to achieve rank 1 by modifying Business Permits only



In [6]:
alternative = "LEG"
target_rank = 2
criteria_permutation =  'Financial Result'

wmsd_transformer.improvement("improvement_single_feature", alternative, target_rank, feature_to_change=criteria_permutation, allow_deterioration=False, popsize=2000, n_generations=200)

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
0,0.0,0.0,0.0,0.0,13.547401


# Lexicographic binary search

In [7]:
criteria_subsets = ['Number of Jobs', 'Financial Result'], ['Number of Jobs', 'Financial Result', 'Business Permits']
for alternative in ["LEG"]:
    print(f"\nProcessing {alternative}, current position in ranking: {current_rank}")
    for criteria_subset in criteria_subsets:
        print("\n---------------------------------------------")
        current_rank = result_df.loc[alternative, "Rank"]
        for criteria_permutation in list(itertools.permutations(criteria_subset)):
            for target_rank in range(current_rank - 1, 0, -1):
                print(f"{criteria_permutation=}")
                solution = wmsd_transformer.improvement("improvement_features", alternative, target_rank, features_to_change=criteria_permutation)
                if solution is not None:
                    print(f"{alternative} {current_rank} --> {target_rank} changing:")
                    for criterion in criteria_permutation:
                        print(f"{criterion}: {solution[criterion].item():.4f}")
                    print()
                else:
                    print(f"Not possible to achieve rank {target_rank} by modifying {criteria_permutation}")


Processing LEG, current position in ranking: 5

---------------------------------------------
criteria_permutation=('Number of Jobs', 'Financial Result')
LEG 5 --> 4 changing:
Number of Jobs: 10371.6140
Financial Result: 0.0000

criteria_permutation=('Number of Jobs', 'Financial Result')
LEG 5 --> 3 changing:
Number of Jobs: 15432.7232
Financial Result: 0.0000

criteria_permutation=('Number of Jobs', 'Financial Result')
LEG 5 --> 2 changing:
Number of Jobs: 19698.0000
Financial Result: 6.8599

criteria_permutation=('Number of Jobs', 'Financial Result')
Not possible to achieve rank 1 by modifying ('Number of Jobs', 'Financial Result')
criteria_permutation=('Financial Result', 'Number of Jobs')
LEG 5 --> 4 changing:
Financial Result: 3.2531
Number of Jobs: 0.0000

criteria_permutation=('Financial Result', 'Number of Jobs')
LEG 5 --> 3 changing:
Financial Result: 4.6959
Number of Jobs: 0.0000

criteria_permutation=('Financial Result', 'Number of Jobs')
LEG 5 --> 2 changing:
Financial Res

In [8]:
alternative = "LEG"
target_rank = 2
criteria_permutation = ['Number of Jobs', 'Business Permits', 'Financial Result']

wmsd_transformer.improvement("improvement_features", alternative, target_rank, features_to_change=criteria_permutation, allow_deterioration=False, popsize=2000, n_generations=200)

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
0,0.0,0.0,19698.0,182.0,1.54673


# Non-linear programming

In [9]:
alternative = "LEG"
target_rank = 1
criteria_permutation = ['Number of Jobs', 'Business Permits', 'Financial Result']

wmsd_transformer.improvement("improvement_non_linear_programming", alternative, target_rank, features_to_change=criteria_permutation)

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
0,0.0,0.0,12427.393217,107.365929,9.490325


# Retaining WM

In [10]:
alternative = "STA"
target_rank = 7
criteria_permutation = criteria_names

wmsd_transformer.improvement(
    "improvement_non_linear_programming",
    alternative,
    target_rank,
    features_to_change=criteria_permutation,
    constant_WM=True
)


,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
0,-2.113832,0.003268,-20.881441,0.0,0.009775


# Evolutionary search (NSGA-II)


In [11]:
alternative = "LEG"
target_rank = 2
popsize = 1000
n_generations = 200

In [12]:
criteria_permutation = ['Financial Result']

wmsd_transformer.improvement(
    "improvement_genetic", alternative, target_rank, features_to_change=criteria_permutation,
    allow_deterioration=False, popsize=popsize, n_generations=n_generations, save_checkpoints=False
)[0]

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
0,-0.0,-0.0,0.0,0.0,13.547346


In [13]:
criteria_permutation = ['Number of Jobs', 'Business Permits', 'Financial Result']
wmsd_transformer.improvement(
    "improvement_genetic", alternative, target_rank, features_to_change=criteria_permutation,
    allow_deterioration=False, popsize=popsize, n_generations=n_generations, save_checkpoints=False
)[0].sample(10).sort_index()

,Total Area,Capital Expenditures,Number of Jobs,Business Permits,Financial Result
30,-0.0,-0.0,6528.690775,11.932883,9.848312
44,-0.0,-0.0,702.662892,62.235357,9.894672
54,-0.0,-0.0,9231.498316,95.387623,5.858051
67,-0.0,-0.0,323.559041,4.755171,13.117419
127,-0.0,-0.0,8736.675186,119.646806,5.130533
169,-0.0,-0.0,230.100004,140.862543,7.355288
461,-0.0,-0.0,10016.941682,126.869716,4.565039
475,-0.0,-0.0,7887.791573,24.992684,8.790320
798,-0.0,-0.0,12091.754196,176.371400,3.071579
875,-0.0,-0.0,14218.100355,79.823826,4.868782


In [14]:
criteria_permutation = criteria_names
print(
    wmsd_transformer.improvement(
        "improvement_genetic", alternative, target_rank, features_to_change=criteria_permutation,
        allow_deterioration=False, popsize=popsize, n_generations=n_generations, save_checkpoints=False
    )
)

(       Total Area  Capital Expenditures  Number of Jobs  Business Permits  \
0   -8.003056e+02             -0.518007      857.650816          0.683750   
1   -7.988933e+02             -2.162260     1831.218295         36.429337   
2   -7.975934e+02             -1.937724     3293.069535        119.962266   
3   -7.964795e+02             -2.162260    13334.246413        127.578420   
4   -7.961509e+02             -0.500286     4019.822078        169.723512   
..            ...                   ...             ...               ...   
995 -9.678818e-03             -1.008825    13126.984548          2.065696   
996 -7.171164e-03             -2.788675      285.847485          3.016404   
997 -5.376397e-04             -0.097255     1071.714809         23.889200   
998 -5.360345e-10             -0.044341      534.865575         36.711597   
999 -5.360345e-10             -0.008196     1345.656631         36.711597   

     Financial Result  
0           11.721793  
1            8.129806  
2 